In [24]:
import os
import warnings
import sys

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from sklearn import linear_model, preprocessing, metrics, model_selection
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from mlflow.tracking import MlflowClient

import seaborn as sn

if 'inline_rc' not in dir():
    inline_rc = dict(mpl.rcParams)

SEED = 10


# import logging
# logging.basicConfig(level=logging.WARN)
# logger = logging.getLogger(__name__)
np.random.seed(SEED)

# Reset do estilo de cores do matplotlib 

In [25]:
# reset matplotlib

mpl.rcParams.update(inline_rc)
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 14}
mpl.rc('font', **font)
lines = {'linewidth' : 3}
mpl.rc('lines', **lines)

# NBA Kobe Bryant

In [26]:
registered_model_name = 'modelo_kobe_bryant'
min_precision = 0.7
model_version = -1 # recuperar a ultima versao
nexamples = 4

# Experimento de Classificação de Arremessos 

In [27]:
# Para usar o sqlite como repositorio
mlflow.set_tracking_uri("sqlite:///mlruns.db")

experiment_name = 'Classificador de Arremessos'
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
    experiment = mlflow.get_experiment(experiment_id)
experiment_id = experiment.experiment_id

# Leitura dos Dados de Classificação de Arremessos 

In [35]:
df_kobe = pd.read_csv('../Data/dataset_kobe.csv',sep=',')

In [29]:
df_kobe.head()

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
0,Jump Shot,Jump Shot,10,20000012,33.9723,167,72,-118.1028,10,1,...,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,1
1,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,...,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,2
2,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1,...,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,3
3,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1,...,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,4
4,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,6,2,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,5


In [34]:
df_kobe.shape

(30697, 24)

In [30]:
df_kobe.columns

Index(['action_type', 'combined_shot_type', 'game_event_id', 'game_id', 'lat',
       'loc_x', 'loc_y', 'lon', 'minutes_remaining', 'period', 'playoffs',
       'season', 'seconds_remaining', 'shot_distance', 'shot_made_flag',
       'shot_type', 'shot_zone_area', 'shot_zone_basic', 'shot_zone_range',
       'team_id', 'team_name', 'game_date', 'matchup', 'opponent', 'shot_id'],
      dtype='object')

In [36]:
# COLOCAR RUN DE LEITURA DE DADOS
# PARAMETROS: top_features,
# METRICS: SHAPE de cada base de dados
# ARTIFACTS: nenhum

top_features = ['lat','lng','minutes remaining', 'playoffs','shot_distance']

with mlflow.start_run(experiment_id=experiment_id, run_name = 'PreparacaoDados'):
    df_kobe = pd.read_csv('../Data/dataset_kobe.csv',sep=',')
    kobe_target_col = 'shot_made_flag'
    kobe_label_map = df_kobe[['shot_made_flag']].drop_duplicates()
    drop_cols = ['shot_made_flag']
    df_kobe.drop(drop_cols, axis=1, inplace=True)
    df_kobe = df_kobe[top_features + ['shot_type', kobe_target_col]].copy()

    data_kobe = df_kobe[df_kobe['shot_type'] == '2PT Field Goal'].copy().drop('shot_type', axis=1)
    
    # Separar parte para compor a base de operacao
    data_kobe, data_operation, ytrain, ytest = model_selection.train_test_split(data_kobe, 
                                                                            data_kobe[kobe_target_col],
                                                                            test_size=0.2)
    data_kobe[kobe_target_col]      = ytrain
    data_operation[kobe_target_col] = ytest

    # Base de vinhos brancos
    data_novelty = df_kobe[df_kobe['shot_type'] == '3PT Field Goal'].copy().drop('shot_type', axis=1)

    data_novelty.to_parquet('modelo_kobe_novidade.parquet')
    data_operation.to_parquet('modelo_kobe_operacao.parquet')
    
    # LOG DE PARAMETROS DO MODELO
    mlflow.log_param("top_features", top_features)

    # LOG DE METRICAS GLOBAIS
    mlflow.log_metric("data_dev", data_kobe.shape[0])
    mlflow.log_metric("data_operation", data_operation.shape[0])
    mlflow.log_metric("data_novelty", data_novelty.shape[0])
   
    
mlflow.end_run()

print('== Bases de Dados ==')
print(f'data_kobe {data_kobe.shape}')
print(f'data_operation {data_operation.shape}')
print(f'data_novelty {data_novelty.shape}')
print(f'Columns: {data_kobe.columns}')

KeyError: "['lng', 'minutes remaining', 'shot_made_flag'] not in index"